# Tutorial 1: Instatiating a *scenario category*

In this tutorial, we will cover the following items:

1. Create *actor categories*, *activity categories*, and *physical thing categories*
2. Instantiate a *scenario category*
3. Show all tags of the *scenario category*
4. Use the `includes` function of a *scenario category*
5. Export the objects

In [1]:
# Before starting, let us do the necessary imports
import os
import json
from domain_model import ActorCategory, ActivityCategory, Constant, ScenarioCategory, \
    Sinusoidal, Spline3Knots, StateVariable, PhysicalElementCategory, Tag, VehicleType, \
    actor_category_from_json, scenario_category_from_json

## 1. Create *actor categories*, *activity categories*, and the *static physical thing categories*

In this tutorial, we will create a *scenario category* in which another vehicle changes lane such that it becomes the ego vehicle's leading vehicle. This is often referred to as a "cut-in scenario". The *scenario category* is depicted in the figure below. Here, the blue car represents the ego vehicle and the red car represents the vehicle that performs the cut in.

<img src="./examples/images/cut-in.png" alt="Cut in" width="400"/>

To create the *scenario category*, we first need to create the *actor categories*, *activity categories*, and the *physical things*. Let us start with the *actor categories*. Just like most objects, an *actor category* has a `name`, a `uid` (a unique ID), and `tags`. Additionally, an *actor category* has a `vehicle_type`. 

In this implementation of the domain model, it is checked whether the correct types are used. For example, `name` must be a string. Similarly, `uid` must be an integer. `tags` must be a (possibly empty) list of type `Tag`. This is to ensure that only tags are chosen out of a predefined list. This is done for consistency, such that, for example, users do not use the tag "braking" at one time and "Braking" at another time. Note, however, that the disadvantage is that it might be very well possible that the list of possible tags is not complete, so if there is a good reason to add a `Tag` this should be allowed. Lastly, the `vehicle_type` must be of type `VehicleType`.

Now let us create the *actor categories*. For this example, we assume that both *actor categories* are "vehicles". Note that we can ignore the `uid` for now. When not `uid` is given, a unique ID is generated automatically. If no `tags` are provided, it will default to an empty list.

In [2]:
EGO_VEHICLE = ActorCategory(VehicleType.Vehicle, name="Ego vehicle", 
                            tags=[Tag.EgoVehicle, Tag.RoadUserType_Vehicle])
TARGET_VEHICLE = ActorCategory(VehicleType.Vehicle, name="Target vehicle",
                               tags=[Tag.RoadUserType_Vehicle])

It is as simple as that. If it does not throw an error, you can be assured that a correct *actor category* is created. For example, if we would forget to add the brackets around the `Tag.RoadUserType_Vehicle` - such that it would not be a *list* of `Tag` - an error will be thrown:

In [3]:
# The following code results in an error! 
# The error is captured as to show only the final error message.
try:
    ActorCategory(VehicleType.Vehicle, name="Target vehicle", tags=Tag.RoadUserType_Vehicle)
except TypeError as error:
    print(error)

Input 'tags' should be of type typing.List but is of type <enum 'Tag'>.


Now let us create the *activity categories*:

In [4]:
FOLLOWING_LANE = ActivityCategory(Constant(), StateVariable.LATERAL_POSITION,
                                  name="Following lane",
                                  tags=[Tag.VehicleLateralActivity_GoingStraight])
CHANGING_LANE = ActivityCategory(Sinusoidal(), StateVariable.LATERAL_POSITION,
                                 name="Changing lane",
                                 tags=[Tag.VehicleLateralActivity_ChangingLane])
DRIVING_FORWARD = ActivityCategory(Spline3Knots(), StateVariable.SPEED,
                                   name="Driving forward",
                                   tags=[Tag.VehicleLongitudinalActivity_DrivingForward])

The last object we need to define before we can define the *scenario category* is the *static physical thing category*. A *scenario category* may contain multiple *physical things*, but for now we only define one that specifies the road layout. We assume that the scenario takes place at a straight motorway with multiple lanes:

In [5]:
MOTORWAY = PhysicalElementCategory(description="Motorway with multiple lanes",
                                   name="Motorway",
                                   tags=[Tag.RoadLayout_Straight,
                                         Tag.RoadType_PrincipleRoad_Motorway])

## 2. Instantiate a *scenario category*

To define a *scenario category*, we need a description and a location to an image. After this, the static content of the scenario can be specified using the `set_physical_things` function. Next, to describe the dynamic content of the scenarios, the *actor categories* can be passed using the `set_actors` function and the *activity categories* can be passed using the `set_activities` function. Finally, using `set_acts`, it is described which activity is connected to which actor. 

Note: It is possible that two actors perform the same activity. In this example, both the ego vehicle and the target vehicle are driving forward.

In [6]:
CUTIN = ScenarioCategory("./examples/images/cut-in.png",
                         description="Cut-in at the motorway",
                         name="Cut-in")
CUTIN.set_physical_elements([MOTORWAY])
CUTIN.set_actors([EGO_VEHICLE, TARGET_VEHICLE])
CUTIN.set_activities([FOLLOWING_LANE, CHANGING_LANE, DRIVING_FORWARD])
CUTIN.set_acts([(EGO_VEHICLE, DRIVING_FORWARD), (EGO_VEHICLE, FOLLOWING_LANE),
                (TARGET_VEHICLE, DRIVING_FORWARD), (TARGET_VEHICLE, CHANGING_LANE)])

## 3. Show all tags of the *scenario category*

The tags should be used to define the *scenario category* in such a manner that also a computer can understand. However, we did not pass any tags to the *scenario category* itself. On the other hand, the attributes of the *scenario category* (in this case, the *physical things*, the *activity categories*, and the *actor categories*) have tags. Using the `derived_tags` function of the *scenario category*, these tags can be retrieved.

Running the `derived_tags` function returns a dictionary with (key,value) pairs. Each key is formatted as `<name>::<class>` and the corresponding value contains a list of tags that are associated to that particular object. For example, `Ego vehicle::ActorCategory` is a key and the corresponding tags are the tags that are passed when instantiating the ego vehicle (`EgoVehicle`) and the tags that are part of the *activity categories* that are connected with the ego vehicle (`GoingStraight` and `DrivingForward`).

In [7]:
CUTIN.derived_tags()

{'Ego vehicle::ActorCategory': [<Tag.EgoVehicle: 13>,
  <Tag.RoadUserType_Vehicle: 1>,
  <Tag.VehicleLongitudinalActivity_DrivingForward: 24>,
  <Tag.VehicleLateralActivity_GoingStraight: 14>],
 'Target vehicle::ActorCategory': [<Tag.RoadUserType_Vehicle: 1>,
  <Tag.VehicleLongitudinalActivity_DrivingForward: 24>,
  <Tag.VehicleLateralActivity_ChangingLane: 15>],
 'Motorway::PhysicalElementCategory': [<Tag.RoadLayout_Straight: 131>,
  <Tag.RoadType_PrincipleRoad_Motorway: 65>]}

Another way - and possibly easier way - to show the tags, is to simply print the scenario category. Doing this will show the name, the description, and all tags of the scenario category.

In [8]:
print(CUTIN)

Name: Cut-in
Description:
  Cut-in  at  the  motorway
Tags:
├─ Ego vehicle::ActorCategory
│  ├─ EgoVehicle
│  ├─ RoadUserType_Vehicle
│  ├─ VehicleLongitudinalActivity_DrivingForward
│  └─ VehicleLateralActivity_GoingStraight
├─ Target vehicle::ActorCategory
│  ├─ RoadUserType_Vehicle
│  ├─ VehicleLongitudinalActivity_DrivingForward
│  └─ VehicleLateralActivity_ChangingLane
└─ Motorway::PhysicalElementCategory
   ├─ RoadLayout_Straight
   └─ RoadType_PrincipleRoad_Motorway



## 4. Use the *includes* function of a *scenario category*

A *scenario category* A includes another *scenario category* B if it comprises all scenarios that are comprised in B. Loosely said, this means that *scenario category* A is "more general" than *scenario category* B. To demonstrate this, let us first create another *scenario category*. The only different between the following *scenario category* is that the target vehicle comes from the left side of the ego vehicle. This means that the target vehicle performs a right lane change, whereas our previously defined *scenario category* did not define to which side the lane change was.

In [9]:
CHANGING_LANE_RIGHT = ActivityCategory(Sinusoidal(), StateVariable.LATERAL_POSITION,
                                       name="Changing lane right",
                                       tags=[Tag.VehicleLateralActivity_ChangingLane_Right])
CUTIN_LEFT = ScenarioCategory("./examples/images/cut-in.png", 
                              description="Cut-in from the left at the motorway",
                              name="Cut-in from left")
CUTIN_LEFT.set_physical_elements([MOTORWAY])
CUTIN_LEFT.set_actors([EGO_VEHICLE, TARGET_VEHICLE])
CUTIN_LEFT.set_activities([FOLLOWING_LANE, CHANGING_LANE_RIGHT, DRIVING_FORWARD])
CUTIN_LEFT.set_acts([(EGO_VEHICLE, DRIVING_FORWARD), (EGO_VEHICLE, FOLLOWING_LANE),
                     (TARGET_VEHICLE, DRIVING_FORWARD), (TARGET_VEHICLE, CHANGING_LANE_RIGHT)])

To ensure ourselves that we correctly created a new *scenario category*, we can print the scenario category. Note the difference with the previously defined *scenario category*: now the target vehicle performs a right lane change (see the tag `VehicleLateralActivity_ChangingLane_Right`).

In [10]:
print(CUTIN_LEFT)

Name: Cut-in from left
Description:
  Cut-in  from  the  left  at  the  motorway
Tags:
├─ Ego vehicle::ActorCategory
│  ├─ EgoVehicle
│  ├─ RoadUserType_Vehicle
│  ├─ VehicleLongitudinalActivity_DrivingForward
│  └─ VehicleLateralActivity_GoingStraight
├─ Target vehicle::ActorCategory
│  ├─ RoadUserType_Vehicle
│  ├─ VehicleLongitudinalActivity_DrivingForward
│  └─ VehicleLateralActivity_ChangingLane_Right
└─ Motorway::PhysicalElementCategory
   ├─ RoadLayout_Straight
   └─ RoadType_PrincipleRoad_Motorway



Because our original *scenario category* (`CUTIN`) is more general than the *scenario category* we just created (`CUTIN_LEFT`), we expect that `CUTIN` *includes* `CUTIN_LEFT`. In other words: because all "cut ins from the left" are also "cut ins", `CUTIN` *includes* `CUTIN_LEFT`.

The converse is not true: not all "cut ins" are "cut ins from the left". 

Let's check it:

In [11]:
print(CUTIN.includes(CUTIN_LEFT))  # True
print(CUTIN_LEFT.includes(CUTIN))  # False

True
False


## 5. Export the objects

It would be cumbersome if one would be required to define a scenario category each time again. Luckily, there is an easy way to export the objects we have created. 

Each object of this domain model comes with a `to_json` function and a `to_json_full` function. These functions return a dictionary that can be directly written to a .json file. The difference between `to_json` and `to_json_full` is that with `to_json`, rather than also returning the full dictionary of the attributes, only a reference (using the unique ID and the name) is returned. In case of the *physical thing*, *actor category*, and *activity category*, this does not make any difference. For the *scenario category*, however, this makes a difference. 

To see this, let's see what the `to_json` function returns.

In [12]:
CUTIN.to_json()

{'name': 'Cut-in',
 'id': '286256851994652760577561676181519991363',
 'tags': [],
 'description': 'Cut-in at the motorway',
 'image': './examples/images/cut-in.png',
 'physical_element_categories': [{'name': 'Motorway',
   'uid': 136030983822332810271896256250787862498}],
 'actor_categories': [{'name': 'Ego vehicle',
   'uid': 5167070121164500855129477838633744559},
  {'name': 'Target vehicle', 'uid': 120414851362655623839622676889850189939}],
 'activity_categories': [{'name': 'Following lane',
   'uid': 120517073365581952625989367855617702427},
  {'name': 'Changing lane', 'uid': 326032736276892065625470323695875443268},
  {'name': 'Driving forward', 'uid': 275940672025515824349081048301576042412}],
 'acts': [{'actor': 5167070121164500855129477838633744559,
   'activity': 275940672025515824349081048301576042412},
  {'actor': 5167070121164500855129477838633744559,
   'activity': 120517073365581952625989367855617702427},
  {'actor': 120414851362655623839622676889850189939,
   'activity':

As can be seen, the *physical thing category* (see `physical_thing_categories`) only returns the `name` and `uid`. This is not enough information for us if we would like to recreate the *physical thing category*. Therefore, for now we will use the `to_json_full` functionality. 

Note, however, that if we would like to store the objects in a database, it would be better to have separate tables for *scenario categories*, *physical thing categories*, *activity categories*, and *actor categories*. In that case, the `to_json` function becomes handy. We will demonstrate this in a later tutorial.

Also note that Python has more efficient ways to store objects than through some json code. However, the reason to opt for the current approach is that this would be easily implementable in a database, such that it is easily possible to perform queries on the data. Again, the actual application of this goes beyond the current tutorial.

To save the returned dictionary to a .json file, we will use the external library `json`.

In [13]:
FILENAME = os.path.join("examples", "cutin_qualitative.json")
with open(FILENAME, "w") as FILE:
    json.dump(CUTIN.to_json_full(), FILE, indent=4)

Let us also save the other *scenario category*, such that we can use it for a later tutorial.

In [14]:
FILENAME_CUTIN_LEFT = os.path.join("examples", "cutin_left_qualitative.json")
with open(FILENAME_CUTIN_LEFT, "w") as FILE:
    json.dump(CUTIN_LEFT.to_json_full(), FILE, indent=4)

So how can we use this .json code to create the *scenario category*? As each object has a `to_json_full` function, for each object there is a `<class_name>_from_json` function. For the objects discussed in this toturial, we have:

- for a *physical thing category*: `physical_thing_category_from_json`
- for an *actor category*: `actor_category_from_json`
- for an *activity category*: `actvitiy_category_from_json`
- for a *model*: `model_from_json`
- for a *scenario category*: `scenario_category_from_json`

Each of these functions takes as input a dictionary that could be a potential output of its corresponding `to_json_full` function. 

To demonstrate this, let's load the just created .json file and see if we can create a new *scenario category* from this.

In [15]:
with open(FILENAME, "r") as FILE:
    CUTIN2 = scenario_category_from_json(json.load(FILE))

To see that this returns a similar *scenario category* as our previously created `CUTIN`, we can print the just created scenario category:

In [16]:
print(CUTIN2)

Name: Cut-in
Description:
  Cut-in  at  the  motorway
Tags:
├─ Ego vehicle::ActorCategory
│  ├─ EgoVehicle
│  ├─ RoadUserType_Vehicle
│  ├─ VehicleLongitudinalActivity_DrivingForward
│  └─ VehicleLateralActivity_GoingStraight
├─ Target vehicle::ActorCategory
│  ├─ RoadUserType_Vehicle
│  ├─ VehicleLongitudinalActivity_DrivingForward
│  └─ VehicleLateralActivity_ChangingLane
└─ Motorway::PhysicalElementCategory
   ├─ RoadLayout_Straight
   └─ RoadType_PrincipleRoad_Motorway



Note that although the just created *scenario category* is now similar to `CUTIN`, it is a different object in Python. That is, if we would change `CUTIN2`, that change will not apply to `CUTIN`.

You reached the end of the first tutorial. In the [next tutorial](./Tutorial%202%20Scenario.ipynb), we will see how we can instantiate a *scenario*.